# Watson API Demo
This notebook demonastrate the invocation of Watson API for training and execution. 
By managing the training process programmatically, the intent recognition performance can be reliably tested with a truly blind test set.

In [1]:
# install watson-developer-cloud library
!pip install --upgrade watson-developer-cloud

Requirement already up-to-date: watson-developer-cloud in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sa7d-9dd5fbf856b6db-aa3b21eda791/.local/lib/python3.5/site-packages
Requirement already up-to-date: pysolr<4.0,>=3.3 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sa7d-9dd5fbf856b6db-aa3b21eda791/.local/lib/python3.5/site-packages (from watson-developer-cloud)
Requirement already up-to-date: requests<3.0,>=2.0 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sa7d-9dd5fbf856b6db-aa3b21eda791/.local/lib/python3.5/site-packages (from watson-developer-cloud)
Requirement already up-to-date: pyOpenSSL>=16.2.0 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sa7d-9dd5fbf856b6db-aa3b21eda791/.local/lib/python3.5/site-packages (from watson-developer-cloud)
Requirement already up-to-date: urllib3<1.23,>=1.21.1 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sa7d-9dd5fbf856b6db-aa3b21eda791/.local/lib/python3.5/site-packages (from requests<3.0,>=2.0->watson-developer-cloud)
Requirement alre

In [2]:
# install sklearn (aka SciKit-Learn)
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sa7d-9dd5fbf856b6db-aa3b21eda791/.local/lib/python3.5/site-packages
Requirement already up-to-date: scikit-learn in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sa7d-9dd5fbf856b6db-aa3b21eda791/.local/lib/python3.5/site-packages (from sklearn)


In [3]:
# import prerequisite libraries
import pandas as pd
import numpy as np
from bokeh.charts import Histogram, output_file, show
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
# The code was removed by DSX for sharing.

,example,intent
0,you can say what is php,capabilities
1,you can not change yourself?,capabilities
2,why should I use you,capabilities
3,where should I turn,capabilities
4,where is the poop deck on this ship,capabilities


In [6]:
# store into variable df
try:
    df = df_data_1
except NameError as e:
    print('Error: Setup is incorrect or incomplete.\n')
    print('Follow the instructions to insert the pandas DataFrame above, and edit to')
    print('make the generated df_data_# variable match the variable used here.')
    raise

In [7]:
# split data into training and test sets
# typically training data is 80%. However, we should investigate this and confirm leading practices
train, test = train_test_split(df, test_size = 0.2)

In [8]:
# supply workspace and authentication parameters
CONVERSATION_USERNAME = '6a6ab898-dea9-423c-baf1-bd6e87ee5dc3'
CONVERSATION_PASSWORD = 'Lb7eIkjJI5Bb'
VERSION = '2017-05-26'
WORKSPACE_ID = 'b1f9955f-c7db-4298-90fe-c98a65735832'

In [9]:
# create the authtication json object
import json
from watson_developer_cloud import ConversationV1
conversation = ConversationV1(
    username=CONVERSATION_USERNAME,
    password=CONVERSATION_PASSWORD,
    version= VERSION
)

In [10]:
# test the API - retrieve intents
intents = conversation.list_intents(WORKSPACE_ID)
print(json.dumps(intents, indent=2))

{
  "pagination": {
    "refresh_url": "/v1/workspaces/b1f9955f-c7db-4298-90fe-c98a65735832/intents?version=2017-05-26"
  },
  "intents": [
    {
      "created": "2017-10-04T07:34:35.355Z",
      "updated": "2017-10-04T07:43:14.700Z",
      "intent": "capabilities",
      "description": "capabilities"
    },
    {
      "created": "2017-10-04T07:34:35.665Z",
      "updated": "2017-10-04T07:43:13.334Z",
      "intent": "interface_issues",
      "description": "interface_issues"
    },
    {
      "created": "2017-10-04T07:34:35.513Z",
      "updated": "2017-10-04T07:43:15.387Z",
      "intent": "locate_amenity",
      "description": "locate_amenity"
    }
  ]
}


In [11]:
# test the API - create intents
create = conversation.create_intent(WORKSPACE_ID,'sample','This is an example')
print(json.dumps(create, indent=2))

{
  "created": "2017-10-05T01:39:25.570Z",
  "updated": "2017-10-05T01:39:25.570Z",
  "intent": "sample",
  "description": "This is an example"
}


In [12]:
#Clear the workspace of all existing intents
intents = conversation.list_intents(workspace_id=WORKSPACE_ID)['intents']
for intent in intents:
    conversation.delete_intent(workspace_id=WORKSPACE_ID, intent=intent['intent'])

In [13]:
# now create intent for each intent in the the training data
for intent in set([x for x in train['intent']]):
    conversation.create_intent(workspace_id=WORKSPACE_ID, intent=intent, description=intent)

In [14]:
# now add examples
for training_data in [x[1] for x in train[:].iterrows()]:
    conversation.create_example(workspace_id=WORKSPACE_ID, intent=training_data.intent, text=training_data.example)

In [15]:
# now test with the test data
results = []
for test_data in [x[1] for x in test[:].iterrows()]:
    try:
        results.append(1 if conversation.message(workspace_id=WORKSPACE_ID, message_input={"text": test_data.example})['intents'][0]['intent'] == test_data.intent else 0)
    except:
        results.append(0)
results = np.array(results)

print("Intent Recognizer Performance: {:.2%}".format(np.sum(results) / results.size))

Intent Recognizer Performance: 94.29%
